In [1]:
# -*- coding: utf-8 -*-
import os
import gc
import argparse
import json
import random
import math
import random
from functools import reduce
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.model_selection import train_test_split, ShuffleSplit, StratifiedShuffleSplit, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_recall_fscore_support, classification_report
import torch
from torch import nn
from torch.optim import Adam, SGD, AdamW
from torch.nn import functional as F
from torch.optim.lr_scheduler import StepLR, CosineAnnealingWarmRestarts, CyclicLR
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
from tqdm import tqdm

from performer_pytorch import PerformerLM
import scanpy as sc
import anndata as ad
from utils import *
from datetime import datetime
from time import time
import torch.multiprocessing as mp
from torch.nn.modules.utils import consume_prefix_in_state_dict_if_present
from torch.utils.tensorboard import SummaryWriter

from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

/opt/conda/rpeyser/envs/rna-pretrain/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
SEED = 2021
NREPS = 3
SAMPLING_FRACS = [1.0, 0.75, 0.5]

In [35]:
# Control sources of randomness
torch.manual_seed(SEED)
random.seed(SEED)

In [36]:
#read Zheng data
zheng_data = sc.read_h5ad("/data/rna_rep_learning/scBERT/Zheng68K.h5ad")
zheng_data

AnnData object with n_obs × n_vars = 68450 × 16906
    obs: 'TSNE.1', 'TSNE.2', 'celltype', 'n_genes'
    uns: 'log1p'

In [37]:
data = zheng_data.X
label = zheng_data.obs.celltype

In [45]:
for k in np.arange(NREPS):
    for frac in SAMPLING_FRACS:
        print("frac {}, rep {}".format(frac, k))
        #downsample training set
        sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=k) #same train/val set split for each frac in k
        for index_train, index_val in sss.split(data, label):
            np.random.seed(k)
            index_train_small = np.random.choice(index_train, round(index_train.shape[0]*frac), replace=False)
            X_train, y_train = data[index_train_small], label[index_train_small]
            X_test, y_test = data[index_val], label[index_val]

        print("Loaded data...")

        #train on train_dataset
        
        #hyperparameter tune using k-fold val on training data
        cv_results = {}
        for c in [1e-3, 1e-2, 1e-1, 1]:
            print("c={}".format(c))
            lr = LogisticRegression(random_state=0, penalty="l1", C=c, solver="liblinear")
            res = cross_validate(lr, X_train, y_train, scoring=['accuracy'])
            cv_results[c] = np.mean(res['test_accuracy'])
        print(cv_results)

        #choose best c and calc performance on val_dataset
        best_ind = np.argmax(list(cv_results.values()))
        c = list(cv_results.keys())[best_ind]
        print("best c={}".format(c))
        lr = LogisticRegression(random_state=0, penalty="l1", C=c, solver="liblinear")
        lr.fit(X_train, y_train)
        print("train set accuracy: " + str(np.around(lr.score(X_train, y_train), 4)))
        print("test set accuracy: " + str(np.around(lr.score(X_test, y_test), 4)))
        val_macro_f1 = sklearn.metrics.f1_score(y_test, lr.predict(X_test), average="macro")
        print("test set macro F1: " + str(np.around(val_macro_f1, 4)))
        


frac 1.0, rep 0
Loaded data...
c=0.001
c=0.01
c=0.1
c=1
{0.001: 0.6885135135135135, 0.01: 0.7837837837837838, 0.1: 0.8014061358655953, 1: 0.7621621621621621}
best c=0.001
train set accuracy: 0.6993


NameError: name 'y_test' is not defined

In [59]:
for k in np.arange(NREPS):
    for frac in SAMPLING_FRACS:
        print("frac {}, rep {}".format(frac, k))
        if k==0 and frac==1.0:
            continue
        #downsample training set
        sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=k) #same train/val set split for each frac in k
        for index_train, index_val in sss.split(data, label):
            np.random.seed(k)
            index_train_small = np.random.choice(index_train, round(index_train.shape[0]*frac), replace=False)
            X_train, y_train = data[index_train_small], label[index_train_small]
            X_test, y_test = data[index_val], label[index_val]

        print("Loaded data...")

        #train on train_dataset
        
        #hyperparameter tune using k-fold val on training data
        cv_results = {}
        for c in [1e-3, 1e-2, 1e-1, 1]:
            print("c={}".format(c))
            lr = LogisticRegression(random_state=0, penalty="l1", C=c, solver="liblinear")
            res = cross_validate(lr, X_train, y_train, scoring=['accuracy'])
            cv_results[c] = np.mean(res['test_accuracy'])
        print(cv_results)

        #choose best c and calc performance on val_dataset
        best_ind = np.argmax(list(cv_results.values()))
        c = list(cv_results.keys())[best_ind]
        print("best c={}".format(c))
        lr = LogisticRegression(random_state=0, penalty="l1", C=c, solver="liblinear")
        lr.fit(X_train, y_train)
        print("train set accuracy: " + str(np.around(lr.score(X_train, y_train), 4)))
        print("test set accuracy: " + str(np.around(lr.score(X_test, y_test), 4)))
        val_macro_f1 = f1_score(y_test, lr.predict(X_test), average="macro")
        print("test set macro F1: " + str(np.around(val_macro_f1, 4)))
        


frac 0.75, rep 0
Loaded data...
c=0.001
c=0.01
c=0.1
c=1
{0.001: 0.6780618456294131, 0.01: 0.7754808862916971, 0.1: 0.7914049184319454, 1: 0.7393474555636718}
best c=0.1
train set accuracy: 0.9522
test set accuracy: 0.7981


NameError: name 'sklearn' is not defined

In [ ]:
nreps_record = []
fracs_record = []
c_record = []
train_acc_record = []
test_acc_record = []
test_macrof1_record = []
for k in np.arange(NREPS):
    for frac in SAMPLING_FRACS:
        nreps_record.append(k)
        print("frac {}, rep {}".format(frac, k))
        fracs_record.append(frac)
        if k==0 and frac==1.0:
            continue
        if k==0 and frac==0.75:
            continue
        #downsample training set
        sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=k) #same train/val set split for each frac in k
        for index_train, index_val in sss.split(data, label):
            np.random.seed(k)
            index_train_small = np.random.choice(index_train, round(index_train.shape[0]*frac), replace=False)
            X_train, y_train = data[index_train_small], label[index_train_small]
            X_test, y_test = data[index_val], label[index_val]

        print("Loaded data...")

        #train on train_dataset
        
        #hyperparameter tune using k-fold val on training data
        cv_results = {}
        for c in [1e-3, 1e-2, 1e-1, 1]:
            print("c={}".format(c))
            lr = LogisticRegression(random_state=0, penalty="l1", C=c, solver="liblinear")
            res = cross_validate(lr, X_train, y_train, scoring=['accuracy'])
            cv_results[c] = np.mean(res['test_accuracy'])
        print(cv_results)

        #choose best c and calc performance on val_dataset
        best_ind = np.argmax(list(cv_results.values()))
        c = list(cv_results.keys())[best_ind]
        c_record.append(c)
        print("best c={}".format(c))
        lr = LogisticRegression(random_state=0, penalty="l1", C=c, solver="liblinear")
        lr.fit(X_train, y_train)
        print("train set accuracy: " + str(np.around(lr.score(X_train, y_train), 4)))
        print("test set accuracy: " + str(np.around(lr.score(X_test, y_test), 4)))
        val_macro_f1 = f1_score(y_test, lr.predict(X_test), average="macro")
        print("test set macro F1: " + str(np.around(val_macro_f1, 4)))
        train_acc_record.append(np.around(lr.score(X_train, y_train), 4))
        test_acc_record.append(np.around(lr.score(X_test, y_test), 4))
        test_macrof1_record.append(np.around(val_macro_f1, 4))
        


frac 1.0, rep 0
frac 0.75, rep 0
frac 0.5, rep 0
Loaded data...
c=0.001
c=0.01
c=0.1
c=1


In [63]:
print(k)
print(frac)
print(c)
print(lr.score(X_test, y_test))

2
0.5
0.1
0.7831263696128561


In [65]:
nreps_record = []
fracs_record = []
c_record = []
train_acc_record = []
test_acc_record = []
test_macrof1_record = []

NREPS=1
SAMPLING_FRACS=[0.25,0.1]

for k in np.arange(NREPS):
    for frac in SAMPLING_FRACS:
        nreps_record.append(k)
        print("frac {}, rep {}".format(frac, k))
        fracs_record.append(frac)
        if k==0 and frac==1.0:
            continue
        if k==0 and frac==0.75:
            continue
        #downsample training set
        sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=k) #same train/val set split for each frac in k
        for index_train, index_val in sss.split(data, label):
            np.random.seed(k)
            index_train_small = np.random.choice(index_train, round(index_train.shape[0]*frac), replace=False)
            X_train, y_train = data[index_train_small], label[index_train_small]
            X_test, y_test = data[index_val], label[index_val]

        print("Loaded data...")

        #train on train_dataset
        
        #hyperparameter tune using k-fold val on training data
        cv_results = {}
        for c in [1e-3, 1e-2, 1e-1, 1]:
            print("c={}".format(c))
            lr = LogisticRegression(random_state=0, penalty="l1", C=c, solver="liblinear")
            res = cross_validate(lr, X_train, y_train, scoring=['accuracy'])
            cv_results[c] = np.mean(res['test_accuracy'])
        print(cv_results)

        #choose best c and calc performance on val_dataset
        best_ind = np.argmax(list(cv_results.values()))
        c = list(cv_results.keys())[best_ind]
        c_record.append(c)
        print("best c={}".format(c))
        lr = LogisticRegression(random_state=0, penalty="l1", C=c, solver="liblinear")
        lr.fit(X_train, y_train)
        print("train set accuracy: " + str(np.around(lr.score(X_train, y_train), 4)))
        print("test set accuracy: " + str(np.around(lr.score(X_test, y_test), 4)))
        val_macro_f1 = f1_score(y_test, lr.predict(X_test), average="macro")
        print("test set macro F1: " + str(np.around(val_macro_f1, 4)))
        train_acc_record.append(np.around(lr.score(X_train, y_train), 4))
        test_acc_record.append(np.around(lr.score(X_test, y_test), 4))
        test_macrof1_record.append(np.around(val_macro_f1, 4))
        


frac 0.25, rep 0
Loaded data...
c=0.001
c=0.01
c=0.1
c=1
{0.001: 0.6259313367421475, 0.01: 0.7357195032870709, 0.1: 0.7432432432432432, 1: 0.7035792549306062}
best c=0.1
train set accuracy: 0.9736
test set accuracy: 0.7528
test set macro F1: 0.6355
frac 0.1, rep 0
Loaded data...
c=0.001
c=0.01
c=0.1
c=1
{0.001: 0.5135149818351499, 0.01: 0.6961260540612605, 0.1: 0.7094595540445955, 1: 0.6765881745158817}
best c=0.1
train set accuracy: 0.9792
test set accuracy: 0.7192
test set macro F1: 0.5992


# logistic regression model

In [7]:
cv_results = {}
for c in [1e-6, 1e-4, 1e-2, 1]:
    cv_results[str(c)] = {}
    lr = LogisticRegression(random_state=0, penalty="l1", C=c, solver="liblinear")
    res = cross_validate(lr, X_train, y_train, scoring=['accuracy','f1_macro'])
    cv_results[c]['test_accuracy'] = np.mean(res['test_accuracy'])


In [8]:
cv_results

{'1e-06': {'test_accuracy': 0.04158144631117604,
  'test_f1_macro': 0.007258445949778784},
 '0.0001': {'test_accuracy': 0.5174762600438276,
  'test_f1_macro': 0.20680632503188856},
 '0.01': {'test_accuracy': 0.7841672753834917,
  'test_f1_macro': 0.6449686172460548},
 '1': {'test_accuracy': 0.7628195763330898,
  'test_f1_macro': 0.6593903887882162}}

In [6]:
cv_results = {}
for c in [1e-3, 1e-1]:
    cv_results[str(c)] = {}
    lr = LogisticRegression(random_state=0, penalty="l1", C=c, solver="liblinear")
    res = cross_validate(lr, X_train, y_train, scoring=['accuracy','f1_macro'])
    cv_results[str(c)]['test_accuracy'] = np.mean(res['test_accuracy'])
    cv_results[str(c)]['test_f1_macro'] = np.mean(res['test_f1_macro'])
cv_results

{'0.001': {'test_accuracy': 0.6888787436084733,
  'test_f1_macro': 0.48298639855688597},
 '0.1': {'test_accuracy': 0.800018261504748,
  'test_f1_macro': 0.6919380706270787}}

In [5]:
#based on cv results
c = 0.1
lr = LogisticRegression(random_state=0, penalty="l1", C=c, solver="liblinear")
lr.fit(X_train, y_train)
lr.score(X_test, y_test)


0.8116143170197224

In [8]:
sklearn.metrics.f1_score(y_test, lr.predict(X_test), average="macro")

0.7074023856802742

In [6]:
#check accuracy per tissue class
for ct in np.unique(zheng_data.obs.celltype):
    print(ct+": {}".format(lr.score(X_test[y_test==ct], y_test[y_test==ct])))

CD14+ Monocyte: 0.8982456140350877
CD19+ B: 0.8382978723404255
CD34+: 0.8125
CD4+ T Helper2: 0.05
CD4+/CD25 T Reg: 0.6814874696847211
CD4+/CD45RA+/CD25- Naive T: 0.4732620320855615
CD4+/CD45RO+ Memory: 0.4297385620915033
CD56+ NK: 0.9264957264957265
CD8+ Cytotoxic T: 0.8075162611418936
CD8+/CD45RA+ Naive Cytotoxic: 0.8993691799339141
Dendritic: 0.7613365155131265
